<a href="https://colab.research.google.com/github/Doutei25/WiDS2026DATATHON-COMPETITION/blob/main/WiDS_2026_Survival_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikit-survival lifelines


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.0/350.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 9.5 MB/s eta 0:00:00


In [2]:
import sksurv
import lifelines

print("sksurv version:", sksurv.__version__)
print("lifelines version:", lifelines.__version__)


sksurv version: 0.27.0
lifelines version: 0.30.1


In [3]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


DATASET_LOADED

In [4]:

import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/ALBERT BETTER/WiDSWorldWide_GlobalDathon26/train.csv')
test = pd.read_csv("/content/drive/MyDrive/ALBERT BETTER/WiDSWorldWide_GlobalDathon26/test.csv")


In [5]:
print(train.shape, test.shape)
train.head()



(221, 37) (95, 35)


,event_id,num_perimeters_0_5h,dt_first_last_0_5h,low_temporal_resolution_0_5h,area_first_ha,area_growth_abs_0_5h,area_growth_rel_0_5h,area_growth_rate_ha_per_h,log1p_area_first,log1p_growth,...,dist_fit_r2_0_5h,alignment_cos,alignment_abs,cross_track_component,along_track_speed,event_start_hour,event_start_dayofweek,event_start_month,time_to_hit_hours,event
0,10892457,3,4.265188,0,79.696304,2.875935,0.036086,0.674281,4.390693,1.354787,...,0.886373,-0.054649,0.054649,-1.937219,-0.106026,19,4,5,18.892512,0
1,11757157,2,1.169918,0,8.946749,0.000000,0.000000,0.000000,2.297246,0.000000,...,0.000000,-0.568898,0.568898,-0.000000,-0.000000,4,4,6,22.048108,1
2,11945086,4,4.777526,0,106.482638,0.000000,0.000000,0.000000,4.677329,0.000000,...,0.000000,0.882385,0.882385,0.000000,0.000000,22,4,8,0.888895,1
3,12044083,1,0.000000,1,67.631125,0.000000,0.000000,0.000000,4.228746,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20,5,8,60.953021,0
4,12052347,2,4.975273,0,35.632874,0.000000,0.000000,0.000000,3.600946,0.000000,...,0.000000,0.934634,0.934634,-0.000000,0.000000,21,5,7,44.990274,0


In [6]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   event_id                      221 non-null    int64  
 1   num_perimeters_0_5h           221 non-null    int64  
 2   dt_first_last_0_5h            221 non-null    float64
 3   low_temporal_resolution_0_5h  221 non-null    int64  
 4   area_first_ha                 221 non-null    float64
 5   area_growth_abs_0_5h          221 non-null    float64
 6   area_growth_rel_0_5h          221 non-null    float64
 7   area_growth_rate_ha_per_h     221 non-null    float64
 8   log1p_area_first              221 non-null    float64
 9   log1p_growth                  221 non-null    float64
 10  log_area_ratio_0_5h           221 non-null    float64
 11  relative_growth_0_5h          221 non-null    float64
 12  radial_growth_m               221 non-null    float64
 13  radia

In [8]:
# Load data
data_path = '/content/drive/MyDrive/ALBERT BETTER/WiDSWorldWide_GlobalDathon26/'
train_df = pd.read_csv(data_path + "train.csv")
test_df  = pd.read_csv(data_path + "test.csv")

print(train_df.shape)
print(test_df.shape)

(221, 37)
(95, 35)


In [9]:
# Basic validation
assert "event" in train_df.columns
assert "time_to_hit_hours" in train_df.columns
assert "event_id" in train_df.columns


**Schema & Integrity Validation**

In [10]:
# event_id uniqueness
assert train_df["event_id"].is_unique
assert test_df["event_id"].is_unique

# Missing values
assert train_df.isnull().sum().sum() == 0
assert test_df.isnull().sum().sum() == 0


In [11]:
# Correct dtypes
train_df["event"] = train_df["event"].astype(bool)
train_df["time_to_hit_hours"] = train_df["time_to_hit_hours"].astype(float)


**Separate Features and Targets**

In [12]:
TARGET_TIME = "time_to_hit_hours"
TARGET_EVENT = "event"
ID_COL = "event_id"

X_train_raw = train_df.drop(columns=[TARGET_TIME, TARGET_EVENT, ID_COL])
X_test_raw  = test_df.drop(columns=[ID_COL])

y_time = train_df[TARGET_TIME].values
y_event = train_df[TARGET_EVENT].values


**Survival Target Construction**

In [13]:
from sksurv.util import Surv

y_train_struct = Surv.from_arrays(
    event=y_event,
    time=y_time
)

y_train_struct[:5]


array([(False, 18.89251185), ( True, 22.04810801), ( True,  0.88889548),
       (False, 60.95302147), (False, 44.99027447)],
      dtype=[('event', '?'), ('time', '<f8')])

**Feature Typing & Grouping**

In [14]:
binary_features = X_train_raw.select_dtypes(include=["bool", "int"]).columns
continuous_features = X_train_raw.select_dtypes(include=["float"]).columns

print("Binary:", len(binary_features))
print("Continuous:", len(continuous_features))


Binary: 5
Continuous: 29


**Outlier Handling**

In [15]:
X_train = X_train_raw.copy()
X_test  = X_test_raw.copy()

for col in continuous_features:
    q1 = X_train[col].quantile(0.01)
    q99 = X_train[col].quantile(0.99)

    X_train[col] = X_train[col].clip(q1, q99)
    X_test[col]  = X_test[col].clip(q1, q99)


**Feature Scaling**

In [16]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[continuous_features] = scaler.fit_transform(X_train[continuous_features])
X_test_scaled[continuous_features]  = scaler.transform(X_test[continuous_features])


**Train / Validation Strategy**

In [17]:
X_train_final = X_train_scaled.values
X_test_final  = X_test_scaled.values


# **SURVIVAL MODELING**

In [18]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

cox_model = CoxPHSurvivalAnalysis()
cox_model.fit(X_train_final, y_train_struct)


/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.35543e-30): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=9.87793e-28): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/usr/local/lib/python3.12/dist-packages/sksurv/linear_model/coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/usr/local/lib/python3.12/dist-packages/sksurv/linear_model/coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=1.73696e-26): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=1.30414e

,alpha,0
,ties,'breslow'
,n_iter,100
,tol,1e-09
,verbose,0


**Cox Proportional** **Hazards**

In [19]:
# C-index
c_index = cox_model.score(X_train_final, y_train_struct)
print("C-index:", round(c_index, 4))


C-index: 0.9559


**Probability Conversion**

In [20]:
surv_functions = cox_model.predict_survival_function(X_train_final)


In [22]:
import numpy as np

# Prediction horizons
prediction_times = np.array([12, 24, 48, 72])

max_time = y_time.max()
prediction_times = prediction_times[prediction_times < max_time]

prediction_times

array([12, 24, 48])

**Build Survival Matrix**

In [23]:
surv_matrix = np.vstack([
    [sf(t) for t in prediction_times]
    for sf in surv_functions
])


**Convert to Event Probabilities + Monotonicity**

In [24]:
event_prob_matrix = 1 - surv_matrix

# Enforce monotonicity
event_prob_matrix = np.maximum.accumulate(event_prob_matrix, axis=1)

# Safety
event_prob_matrix = np.clip(event_prob_matrix, 0, 1)


# **CALIBRATION & METRICS**

In [25]:
from sksurv.metrics import brier_score

times, scores = brier_score(
    y_train_struct,
    y_train_struct,
    surv_matrix,
    prediction_times
)


**Display Metrics**

In [26]:
for t, s in zip(times, scores):
    print(f"Brier score at {int(t)}h: {s:.4f}")

print("Weighted Brier score:", scores.mean())


Brier score at 12h: 0.0584
Brier score at 24h: 0.0354
Brier score at 48h: 0.0232
Weighted Brier score: 0.03896607105284724


# **TEST PREDICTION**

In [27]:
test_surv_functions = cox_model.predict_survival_function(X_test_final)

test_surv_matrix = np.vstack([
    [sf(t) for t in prediction_times]
    for sf in test_surv_functions
])

test_event_probs = 1 - test_surv_matrix
test_event_probs = np.maximum.accumulate(test_event_probs, axis=1)
test_event_probs = np.clip(test_event_probs, 0, 1)


**Submission File**

In [28]:
submission = pd.DataFrame({
    "event_id": test_df["event_id"].values
})

for i, t in enumerate(prediction_times):
    submission[f"prob_{int(t)}h"] = test_event_probs[:, i]

submission.head()


,event_id,prob_12h,prob_24h,prob_48h
0,10662602,0.000000e+00,0.000000e+00,0.000000e+00
1,13353600,4.542947e-01,8.032962e-01,9.149687e-01
2,13942327,3.684764e-08,9.892469e-08,1.499476e-07
3,16112781,3.756474e-01,7.176475e-01,8.529303e-01
4,17132808,1.000000e+00,1.000000e+00,1.000000e+00


In [58]:
submission.to_csv("submission.csv", index=False)

# **Horizon-wise Probability Calibration (Isotonic)**

In [30]:
import numpy as np
import pandas as pd

# Horizons used in the competition
prediction_times = np.array([12, 24, 48])

# Binary labels per horizon
# 1 = event happened before horizon
y_calib = pd.DataFrame({
    "event_id": train["event_id"],
    "y_12h": ((train["event"] == 1) & (train["time_to_hit_hours"] <= 12)).astype(int),
    "y_24h": ((train["event"] == 1) & (train["time_to_hit_hours"] <= 24)).astype(int),
    "y_48h": ((train["event"] == 1) & (train["time_to_hit_hours"] <= 48)).astype(int),
})


**Extract raw CoxPH probabilities (train)**

In [31]:
# From CoxPH
surv_funcs_train = cox_model.predict_survival_function(X_train_final)
cox_train_probs = pd.DataFrame({
    "event_id": train["event_id"],
    "p_12h": 1 - np.array([sf(12) for sf in surv_funcs_train]),
    "p_24h": 1 - np.array([sf(24) for sf in surv_funcs_train]),
    "p_48h": 1 - np.array([sf(48) for sf in surv_funcs_train]),
})

**Fit isotonic calibrators (SAFE)**

In [32]:
from sklearn.isotonic import IsotonicRegression

calibrators = {}

for h in [12, 24, 48]:
    iso = IsotonicRegression(out_of_bounds="clip")
    iso.fit(
        cox_train_probs[f"p_{h}h"],
        y_calib[f"y_{h}h"]
    )
    calibrators[h] = iso


**Apply calibration to TEST predictions**

In [33]:
cox_test_probs = submission.copy()
# columns: event_id, prob_12h, prob_24h, prob_48h


In [34]:
cox_test_calibrated = pd.DataFrame({
    "event_id": cox_test_probs["event_id"],
    "prob_12h": calibrators[12].transform(cox_test_probs["prob_12h"]),
    "prob_24h": calibrators[24].transform(cox_test_probs["prob_24h"]),
    "prob_48h": calibrators[48].transform(cox_test_probs["prob_48h"]),
})


**Enforce monotonicity (FINAL SAFETY)**

In [35]:
def enforce_monotonic(df, cols):
    df = df.copy()
    for i in range(1, len(cols)):
        df[cols[i]] = np.maximum(df[cols[i]], df[cols[i-1]])
    return df

cox_test_calibrated = enforce_monotonic(
    cox_test_calibrated,
    ["prob_12h", "prob_24h", "prob_48h"]
)


# **Model Ensembling**

In [ ]:
# @title
import sksurv.linear_model
print(dir(sksurv.linear_model))

['CoxPHSurvivalAnalysis', 'CoxnetSurvivalAnalysis', 'IPCRidge', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_coxnet', 'aft', 'coxnet', 'coxph']


In [ ]:
# @title
import sksurv.linear_model.aft
print(dir(sksurv.linear_model.aft))

['IPCRidge', 'Ridge', 'SurvivalAnalysisMixin', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'check_array_survival', 'ipc_weights', 'np']


In [ ]:
# @title
!pip install --upgrade scikit-survival

In [ ]:
# @title
import sksurv.linear_model
import sksurv.linear_model.aft

print(f"sksurv version: {sksurv.__version__}")
print("\nContents of sksurv.linear_model:")
for item in dir(sksurv.linear_model):
    if not item.startswith("_"):
        print(f"- {item}")

print("\nContents of sksurv.linear_model.aft:")
for item in dir(sksurv.linear_model.aft):
    if not item.startswith("_"):
        print(f"- {item}")

# Re-attempting the problematic import to show the error again for clarity
try:
    from sksurv.linear_model import WeibullAFTRegression
except ImportError as e:
    print(f"\nFailed to import WeibullAFTRegression from sksurv.linear_model: {e}")

try:
    from sksurv.linear_model.aft import WeibullAFTRegression
except ImportError as e:
    print(f"Failed to import WeibullAFTRegression from sksurv.linear_model.aft: {e}")

print("\nIt appears 'WeibullAFTRegression' is not available in the installed scikit-survival package (0.27.0).")
print("This may be due to a specific build, compatibility issue with Python 3.12, or a change not reflected in standard documentation.")


sksurv version: 0.27.0

Contents of sksurv.linear_model:
- CoxPHSurvivalAnalysis
- CoxnetSurvivalAnalysis
- IPCRidge
- aft
- coxnet
- coxph

Contents of sksurv.linear_model.aft:
- IPCRidge
- Ridge
- SurvivalAnalysisMixin
- check_array_survival
- ipc_weights
- np

Failed to import WeibullAFTRegression from sksurv.linear_model: cannot import name 'WeibullAFTRegression' from 'sksurv.linear_model' (/usr/local/lib/python3.12/dist-packages/sksurv/linear_model/__init__.py)
Failed to import WeibullAFTRegression from sksurv.linear_model.aft: cannot import name 'WeibullAFTRegression' from 'sksurv.linear_model.aft' (/usr/local/lib/python3.12/dist-packages/sksurv/linear_model/aft.py)

It appears 'WeibullAFTRegression' is not available in the installed scikit-survival package (0.27.0).
This may be due to a specific build, compatibility issue with Python 3.12, or a change not reflected in standard documentation.


In [36]:
ensemble = pd.DataFrame({
    "event_id": cox_test_calibrated["event_id"],
    "prob_12h": cox_test_calibrated["prob_12h"],
    "prob_24h": cox_test_calibrated["prob_24h"],
    "prob_48h": cox_test_calibrated["prob_48h"],
})

In [37]:
from lifelines import WeibullAFTFitter

# Prepare the DataFrame for lifelines, combining scaled features with duration and event columns
train_cox = X_train_scaled.copy()
train_cox[TARGET_TIME] = y_time
train_cox[TARGET_EVENT] = y_event

aft = WeibullAFTFitter(
    penalizer=0.01,          # prevents singular matrix
    l1_ratio=0.0
)

aft.fit(
    train_cox,               # features + event + time
    duration_col=TARGET_TIME,
    event_col=TARGET_EVENT
)

<lifelines.WeibullAFTFitter: fitted with 221 total observations, 152 right-censored observations>

In [40]:
# Assuming surv_funcs contains survival functions from your model for the test set
import pandas as pd
import numpy as np

# Explicitly define prediction_times to ensure it's correct
prediction_times = np.array([12, 24, 48])

prob_df = pd.DataFrame()
prob_df['event_id'] = test['event_id']  # make sure this is the test set IDs

for t in prediction_times:
    # Evaluate survival probability at time t for test set
    # Corrected variable name and method for applying to a numpy array
    prob_df[f'prob_{t}h'] = [sf(t) for sf in test_surv_functions]

# Enforce monotonicity (optional)
prob_df = prob_df.sort_values('event_id').reset_index(drop=True)
for i in range(1, len(prediction_times)):
    prob_df[f'prob_{prediction_times[i]}h'] = np.maximum(
        prob_df[f'prob_{prediction_times[i-1]}h'],
        prob_df[f'prob_{prediction_times[i]}h']
    )

# Check shape
print(prob_df.shape)  # should be (95, 4) since prediction_times has 3 elements now

(95, 4)


In [42]:
# -----------------------------
# FINAL SUBMISSION GENERATION
import pandas as pd
import numpy as np

# Prediction horizons in hours
# Use the filtered prediction_times from earlier cells
prediction_times = np.array([12, 24, 48])

# Initialize submission dataframe with test set IDs
submission = pd.DataFrame()
submission['event_id'] = test['event_id']  # test set IDs

# Generate survival probabilities for test set
# Use 'test_surv_functions' and a list comprehension for evaluation
for t in prediction_times:
    submission[f'prob_{t}h'] = [sf(t) for sf in test_surv_functions]

# Enforce monotonicity across horizons
for i in range(1, len(prediction_times)):
    submission[f'prob_{prediction_times[i]}h'] = np.maximum(
        submission[f'prob_{prediction_times[i-1]}h'],
        submission[f'prob_{prediction_times[i]}h']
    )

# Ensure probabilities are within [0, 1]
for col in [f'prob_{t}h' for t in prediction_times]:
    submission[col] = submission[col].clip(0, 1)

# Quick sanity check
print(submission.head())
print("Submission shape:", submission.shape)  # should be (95, 4) if prediction_times has 3 elements

# Save submission file
submission.to_csv('final_submission.csv', index=False)

   event_id  prob_12h  prob_24h  prob_48h
0  10662602  1.000000  1.000000  1.000000
1  13353600  0.545705  0.545705  0.545705
2  13942327  1.000000  1.000000  1.000000
3  16112781  0.624353  0.624353  0.624353
4  17132808  0.000000  0.000000  0.000000
Submission shape: (95, 4)


In [44]:
import pandas as pd
import numpy as np

test_df = pd.read_csv("/content/drive/MyDrive/ALBERT BETTER/WiDSWorldWide_GlobalDathon26/test.csv")

assert "event_id" in test_df.columns
print("Test rows:", len(test_df))

Test rows: 95


In [45]:
HORIZONS = np.array([12, 24, 48, 72])


In [47]:
# X_test must align row-for-row with test_df
X_test = test_df.drop(columns=["event_id"])

# Predict survival functions
surv_funcs = cox_model.predict_survival_function(X_test_final)

In [55]:
probs = []

for sf in surv_funcs:
    # Get the maximum time for which the survival function is defined
    max_defined_time = sf.domain[1]

    surv_vals = []
    for t in HORIZONS: # Using HORIZONS which includes 72
        if t > max_defined_time:
            # If prediction time is beyond the defined domain,
            # use the survival probability at the max defined time.
            # This assumes survival probability plateaus.
            surv_vals.append(sf(max_defined_time))
        else:
            surv_vals.append(sf(t))

    cum_probs = 1 - np.array(surv_vals)
    probs.append(cum_probs)

probs = np.vstack(probs)

In [51]:
probs = np.clip(probs, 0, 1)
probs = np.maximum.accumulate(probs, axis=1)


In [56]:
submission = pd.DataFrame({
    "event_id": test_df["event_id"].values,
    "prob_12h": probs[:, 0],
    "prob_24h": probs[:, 1],
    "prob_48h": probs[:, 2],
    "prob_72h": probs[:, 3]
})

In [59]:
# Column check
expected_cols = ["event_id", "prob_12h", "prob_24h", "prob_48h", "prob_72h"]
assert list(submission.columns) == expected_cols

# Row count check
assert len(submission) == len(test_df)

# Probability bounds
assert ((submission.iloc[:, 1:] >= 0) & (submission.iloc[:, 1:] <= 1)).all().all()

# Monotonicity check
assert (submission["prob_12h"] <= submission["prob_24h"]).all()
assert (submission["prob_24h"] <= submission["prob_48h"]).all()
assert (submission["prob_48h"] <= submission["prob_72h"]).all()

print("✅ Submission PASSES all checks")


✅ Submission PASSES all checks


In [61]:
submission.to_csv("submission.csv", index=False)
print("📤 submission.csv generated")


📤 submission.csv generated
